In [ ]:
import wikipediaapi
import random
from rake_nltk import Rake
r = Rake(language='Dutch', max_length=4)

wiki_wiki = wikipediaapi.Wikipedia(language='nl', user_agent="YourAppName/1.0 (your.email@example.com)")
categories = list(wiki_wiki.page('Categorie:Alles').categorymembers.keys())

results = []

while len(results)<5:
    category = random.choice(categories[1:])
    category_page = wiki_wiki.page(f"{category}")
    def fetch_subcategories(cat):
        member = random.choice(list(cat.categorymembers.items()))[1]
        if member.namespace == wikipediaapi.Namespace.CATEGORY:
            fetch_subcategories(member)
        elif member.namespace == wikipediaapi.Namespace.MAIN:
            results.append(member)
    fetch_subcategories(category_page)

random.shuffle(results)
#denk na over maat voor hoe algemeen een term is. maybe iets van aantal hits, paginalengte, aantal andere talen
for article in results:
    r.extract_keywords_from_text(article.text)
    keywords = r.get_ranked_phrases()
    print(f"\t{article.title}")
    print(f"Some keywords: {keywords[0]}, {keywords[1]}, {keywords[2]}, {keywords[3]}, {keywords[4]}\n")

In [56]:
import wikipediaapi
import random
wiki_wiki = wikipediaapi.Wikipedia(language='en', user_agent="YourAppName/1.0 (your.email@example.com)")
categories = list(wiki_wiki.page('Category:Contents').categorymembers.keys())

def fetch_subcategories(cat):
    member = random.choice(list(cat.categorymembers.items()))[1]
    if member.namespace == wikipediaapi.Namespace.CATEGORY:
        return fetch_subcategories(member)
    elif member.namespace == wikipediaapi.Namespace.MAIN:
        return member


In [57]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def find_relevant_sentences(article_text, eng_sentences, top_n=5):
    # Combine the article keywords with the sentences
    all_texts = [article_text] + eng_sentences['source_sentence'].tolist()

    # Vectorize the text using CountVectorizer (sparse output)
    vectorizer = CountVectorizer(dtype='float32')  # Use float32 for lighter computation
    vectors = vectorizer.fit_transform(all_texts)

    # Compute cosine similarity using sparse matrices
    cosine_sim = cosine_similarity(vectors[0], vectors[1:])

    # Get the top_n most relevant sentences
    top_indices = cosine_sim.argsort(axis=1)[0, -top_n:][::-1]
    top_sentences = eng_sentences.iloc[top_indices]

    return top_sentences

In [62]:
import pandas as pd
# Load English sentences
eng_sentences = pd.read_csv(
    "../tatoeba/eng_sentences.tsv", sep="\t", header=None,
    names=["id", "language", "source_sentence"]
)

In [70]:
# Filter sentences
eng_sentences = eng_sentences[eng_sentences['source_sentence'].str.split().str.len() < 7]

In [71]:
result = None
while result is None:
    result = fetch_subcategories(category_page)

text = result.text

top_sents = find_relevant_sentences(text, eng_sentences)
print(text)
display(top_sents)

Community is an American television sitcom created by Dan Harmon. The show ran for six seasons and 110 episodes, with its first five seasons airing on NBC from September 17, 2009, to April 17, 2014, and the final season airing on Yahoo! Screen from March 17 to June 2, 2015. Set at Greendale Community College, the series stars an ensemble cast playing members of a diverse study group. The main cast includes Joel McHale as Jeff Winger, Gillian Jacobs as Britta Perry, Danny Pudi as Abed Nadir, Yvette Nicole Brown as Shirley Bennett, Alison Brie as Annie Edison, Donald Glover as Troy Barnes, Ken Jeong as Ben Chang, Chevy Chase as Pierce Hawthorne, and Jim Rash as Dean Craig Pelton. A comedy about friendships and relationships, the show uses frequent pop culture references and meta-humor, as well as high-concept episodes including documentary spoofs, action movie homages and storylines inspired by movies.
Despite being critically acclaimed and gaining a cult following, the show faced produc

,id,language,source_sentence
1349875,9011310,eng,The spindle and the distaff.
1799056,11917642,eng,And the details?
145239,434282,eng,At the doctor's and the dentist's.
145210,434216,eng,And the yellow one?
374578,2838114,eng,Remove the screw and the lampshade.


'The Kenora Thistles, officially the Thistles Hockey Club, were a Canadian ice hockey team based in Kenora, Ontario. Founded in 1894, they were originally known as the Rat Portage Thistles. The team competed for the Stanley Cup, the ice hockey championship of Canada, five times between 1903 and 1907. The Thistles won the Cup in January 1907 and defended it once before losing it that March in a challenge series. Composed almost entirely of local players, the team comes from the least populated city to have won the Stanley Cup. Nine players—four of them homegrown—have been inducted into the Hockey Hall of Fame, and the Stanley Cup champion team was inducted into the Northwestern Ontario Sports Hall of Fame.\nThough Kenora is in Ontario, the Thistles competed in Manitoba-based leagues throughout their existence, owing to the city\'s proximity to that province. The team joined the Manitoba Hockey Association (MHA) in 1902, winning the league championship in three of their six seasons. They